# Gravity Modelling Exercise for GEOL 4826/5826 (Part B)

Apply what you learned in Part A to model the gravity data from the Winnipeg River survey. Copy over and adapt code blocks to acheive this.

## Environment setup

Run the following code blocks to import the necessary libraries. This may take some time. 


In [ ]:
# Load packages and libraries

using Pkg
Pkg.status()
Pkg.add("PyPlot")
Pkg.add("NPZ") # Library for loading Numpy files

# Import Libraries
using PyPlot # Plotting
using Base.Iterators # Array operators
using Random;
using NPZ;

# Import User Generated Libraries
include("kernel.jl"); # Forward model library
include("conjugateGradient.jl"); # C.G. solver

# Inversion modeling
Observed gravity data $(d_{obs})$ is provided in a Numpy file below for you. The following block loads it into an array and plots the values along the profile.

In [ ]:
# Provide observed data

d_obs = npzread("WR_gravity_data.npy"); # Load WR Gravity Data
labels = npzread("WR_gravity_data_labels.npy"); # Load station labels for WR Gravity Data

# Plot the values of d_obs using PyPlot
fig = figure(figsize=(16,2))
plot(labels, d_obs)
xlabel("Station No."); ylabel("Gravity (mgal)");
title("Gravity anomaly over profile");
show()

Use the code blocks below to estimate (i.e., invert for) a density model from $d_{obs}$. Make sure to include a plot of  your final model ($m_{est}$) and estimated data ($d_{est}$).

In [ ]:
# Set model extents by creating a blank model

nz = 5; # number of cells in the z direction
nx = 49; # number of cells in x direction
dz = 1125; # meters, cell size in z direction (depth)
dx = 1125; # meters, cell size in x direction (distance)
background = 2.80; # grams/cc, constant starting background density

In [ ]:
# Assemble a sensitivity matrix, or kernel, (G operator) with assuming no change in datum height (z)

M = size(model)[1] * size(model)[2];
N = size(model)[2];

G = generateKernel(M, zeros(N), dx, dz);

(kernel.jl) the model is 5 by 49

In [ ]:
# Set inversion parameters

ζ = 0.10;
β = 0.50;
ε = 0.99;
iterations=5;

In [ ]:
# Get dimensions (M, N) from model and data vectors
M = size(model)[1] * size(model)[2];
N = size(model)[2];

# Generate a G with assuming no change in datum height (z)
G = generateKernel(M, zeros(N), dx, dz);

g_obs = d_obs; # Can add padding here

# Make an H matrix "flatness"
H = Matrix{Float64}(I,M,M) * -1; #smoothness matrix
for i = 1:M-1, j = 2:M
    H[i,i+1] = 1;
end

# Make A matrix [G]
#               [H]

A = hcat(G', ζ * H')';

# Make a P matrix ("hard constraint")
P = Matrix{Float64}(I, M, M);

# Make a Q matrix ("depth weighting matrix")
Q = Matrix{Float32}(I, M, M);

for j = 1:M
    Q_jj = 1 / (((j*dz) + ε)^β);
    Q[j,:] = Q[j,:] * Q_jj;
end

# Make a V matrix ("compactness")
𝓥 = Matrix{Float64}(I,M,M); #initially set to Identity

# Set any initial density anomaly estimates
ρ_back = zeros(M) # Background anomaly is zero

ρ = Array{Float64}(undef,M) .+ ρ_back;
ρ_min = -1;
ρ_max = 1;

null_vector = zeros(M) # Null vector

for i = 1:M # Update hard constraint
    if ρ[i] != 2.5
        P[i,i] = 1e-2
    end
end

(kernel.jl) the model is 5 by 49

In [ ]:
# Inversion iterative loop (minimization of objective function)
for i = 1:iterations
    W = inv(P) * Q * 𝓥
    g_cal = G * ρ
    b = vcat(g_obs - g_cal, null_vector);
    Θ = return_m((A * inv(W)) * transpose((A * inv(W))), b, 40); # C.G.

    global ρ = ρ + (inv(W) * transpose(A * inv(W)) * Θ)

    for j = 1:M
        𝓥[j,j] = 1 / ((ρ[j])^2 + 1e-11)
        if ρ[j] < ρ_min || ρ[j] > ρ_max
            P[j,j] = 1e-2
        else
            P[j,j] = 1
        end
    end
end


d_estf = G * ρ
mse = sum((d_obs .- d_estf).^2) / length(d_obs) # Calculate Mean Squared Error
print("The mean squared error betweent he observed and estimated data is: ", round(mse, digits=5))

# Plot the values of d_est using PyPlot
fig = figure(figsize=(26, 4))
plot(d_estf, label="Estimated Data")
plot(d_obs, "--", label="Observed Data")
xlabel("Station")
ylabel("Gravity (mgal)")
legend()
title("Gravity anomaly over profile")
show()

The mean squared error betweent he observed and estimated data is: 0.00192

PyObject <matplotlib.colorbar.Colorbar object at 0x00000164135CB010>

In [ ]:
# Plot final result (model and estimated data)

model_est = reshape(ρ, (nx, nz)) .+ background

fig = figure(figsize=(26,4))
imshow(transpose(model_est) , vmin=2.67, vmax=3.0)
ylabel("Depth (cells)")
xlabel("Distance along profile (cells)")
title("Estimated Model (grams/cc)")
colorbar(orientation="horizontal")

## End
Created by Christopher Mancuso. 2024.
Rev 1.02